# DataLab Cup 2: CNN for Object Detection

## Dataset: PASCAL VOC2007

In [1]:
classes_name =  ["aeroplane", "bicycle", "bird", "boat", "bottle", 
                 "bus", "car", "cat", "chair", "cow", "diningtable", 
                 "dog", "horse", "motorbike", "person", "pottedplant", 
                 "sheep", "sofa", "train","tvmonitor"]

## Processed data format

In [2]:
training_data_file = open("./pascal_voc_training_data.txt", "r")
for i, line in enumerate(training_data_file):
    if i >5:
        break
    line = line.strip()
    print(line)

000005.jpg 263 211 324 339 8 165 264 253 372 8 5 244 67 374 8 241 194 295 299 8 277 186 312 220 8
000007.jpg 141 50 500 330 6
000009.jpg 69 172 270 330 12 150 141 229 284 14 285 201 327 331 14 258 198 297 329 14
000012.jpg 156 97 351 270 6
000016.jpg 92 72 305 473 1
000017.jpg 185 62 279 199 14 90 78 403 336 12


In [3]:
import tensorflow as tf
import numpy as np
import os
import shutil
from data_aug.data_aug import *
from data_aug.bbox_util import *
from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import cv2

## Hyperparameters

In [4]:
# common params
IMAGE_SIZE = 448
BATCH_SIZE = 4
NUM_CLASSES = 20
MAX_OBJECTS_PER_IMAGE = 20

# dataset params
DATA_PATH = './pascal_voc_training_data.txt'
IMAGE_DIR = './VOCdevkit_train/VOC2007/JPEGImages/'

# combined data path/ combined image directory
COM_DATA_PATH = './pascal_voc_training_data_combined.txt'
COM_IMAGE_DIR = './VOCdevkit_train/VOC2007/JPEGImages_combined/'

# model params
CELL_SIZE = 7
BOXES_PER_CELL = 2
OBJECT_SCALE = 1
NOOBJECT_SCALE = 0.5
CLASS_SCALE = 1
COORD_SCALE = 5

# training params
LEARNING_RATE = 1e-5
EPOCHS = 40

## Check class object numbers

In [5]:
images = [] # [000005.jpg, 000007.jpg, ...] 共4974張
records = [] # [[263,211,324,339,8,165,264,253,372,8], [...], ...]
num_class = [] # [5,1,...]每張圖有多少個class，為了下方擴增dataset時加速。

with open(DATA_PATH, 'r') as f:
    for line in f:
        tokens = line.split()
        images.append(tokens[0])
        # 每筆record放 5*物件個數
        record = list(map(float, tokens[1:])) # [xmin, ymin, xmax, ymax, class] = [263,211,324,339,8, ...] 
    
        # 每筆record最多只塞 MAX_OBJECTS_PER_IMAGE個物件
        if len(record) > MAX_OBJECTS_PER_IMAGE*5:
            record = record[:MAX_OBJECTS_PER_IMAGE*5] 
        records.append(record)
        num_class.append(len(record) // 5)

# 把每筆record(一維)拆成一個一個的物件框框(二維) dim: row=n/5, col=5
# record = [1,2,3,4,5,6,7,8,9,10]
# box = [[1, 2, 3, 4, 5],
#        [6, 7, 8, 9,10]]

boxes_list = []
for record in records:
    boxes = np.array(record)
    boxes = boxes.reshape((-1, 5))
    boxes_list.append(boxes)
    
image2boxes = dict(zip(images, boxes_list))

# classes in record in records
# classes = [[8,8,8],[6]]

classes = []
for record in records:
    record_class = []
    for i in range(4, len(record), 5):
        record_class.append(int(record[i]))
    classes.append(record_class)

# print(len(classes)) # 4974
        
# 計算每個class的數量 
class_cnt = np.zeros(NUM_CLASSES, int)
for record_class in classes: 
    for c in record_class:
        class_cnt[c] += 1
    
for i, num in enumerate(class_cnt):
    print(f'{i:2d}) {classes_name[i]:12}: {num}')

 0) aeroplane   : 331
 1) bicycle     : 412
 2) bird        : 577
 3) boat        : 398
 4) bottle      : 612
 5) bus         : 271
 6) car         : 1634
 7) cat         : 389
 8) chair       : 1423
 9) cow         : 356
10) diningtable : 309
11) dog         : 536
12) horse       : 403
13) motorbike   : 387
14) person      : 5318
15) pottedplant : 603
16) sheep       : 353
17) sofa        : 419
18) train       : 328
19) tvmonitor   : 366


In [6]:
combined_images = []
class_cnt = np.zeros(NUM_CLASSES, int) # index:0-19 value: # of class
num_image = len(images) # 4974
image_no = np.zeros(num_image, int)

# 先將原本的dataset放進combined dataset
# 並計算對應的各class數目
for i in range(num_image):
    record_class = classes[i]
    combined_images.append(f'{image_no[i]}-{images[i]}')
    image_no[i] += 1
    for c in record_class:
        class_cnt[c] += 1
        
_MIN = 5318

# 先看圖片所含的各個class是否都超過_MIN
# 如果其中有一個class不到_MIN我就加進去並更新class_cnt
# It turns out that person, car and chair are still imbalanced.
# while True:
#     for i, v in enumerate(classes):
#         s = set(v)
#         for c in s:
#             if class_cnt[c] < _MIN:
#                 combined_images.append(f'{image_no[c]}-{images[i]}')
#                 image_no += 1
#                 for each_c in v:
#                     class_cnt[each_c] += 1
#     if (class_cnt >= _MIN).all():
#         break

# tag = np.zeros(num_image, int)
# while True:
#     for i, v in enumerate(classes): # v = [8,8,8,8,8]
#         s = set(v)
#         for c in s:
#             if tag[i] or class_cnt[c] >= _MIN:
#                 tag[i] = 1
#                 continue
#             else:
#                 combined_images.append(f'{image_no[c]}-{images[i]}')
#                 image_no += 1
#                 for each_c in v:
#                     class_cnt[each_c] += 1
#     if (class_cnt >= _MIN).all():
#         break

# 先按照每張圖片所包含的class數量排序，並維護好index。
# 再依序看每張圖片所含的各個class是否有任一個超過_MIN。
# 若有，則不加入dataset，並上tag，未來也不加入。
# 否則，加入combined並更新class_cnt。

indexed_list = list(enumerate(num_class))
sorted_indices = sorted(indexed_list, key=lambda x: x[1], reverse=True)
sorted_indices = [index for index, value in sorted_indices]

tag = np.zeros(num_image, int)
while True:
    for i in sorted_indices:
        if tag[i]:
            continue
        record_class = classes[i]
        if np.any(class_cnt[record_class] >= _MIN): # 這張圖片其中一個class已經超過_MIN
            tag[i] = 1 # 未來不再看這張圖片
            continue
        else:
            combined_images.append(f'{image_no[i]}-{images[i]}')
            image_no[i] += 1
            for c in record_class:
                class_cnt[c] += 1
                    
    if np.all(class_cnt >= _MIN):
        break
        
for i, num in enumerate(class_cnt):
    print(f'{i:2d}) {classes_name[i]:12}: {num}')
    
# print(combined_images[0])

 0) aeroplane   : 5318
 1) bicycle     : 5318
 2) bird        : 5319
 3) boat        : 5318
 4) bottle      : 5325
 5) bus         : 5322
 6) car         : 5318
 7) cat         : 5319
 8) chair       : 5323
 9) cow         : 5319
10) diningtable : 5318
11) dog         : 5318
12) horse       : 5318
13) motorbike   : 5318
14) person      : 5318
15) pottedplant : 5322
16) sheep       : 5318
17) sofa        : 5319
18) train       : 5319
19) tvmonitor   : 5319


In [7]:
def gen_combined_dataset():
    if os.path.exists(COM_IMAGE_DIR):
        shutil.rmtree(COM_IMAGE_DIR)

    if not os.path.exists(COM_IMAGE_DIR):
        os.mkdir(COM_IMAGE_DIR)

    with open(COM_DATA_PATH, 'w') as f:
        np.random.seed(514)
        trans = Sequence([# RandomRotate(10), 
                          # RandomTranslate(0.2), 
                          RandomHSV(20, 20, 20), 
                          RandomHorizontalFlip(1), 
                          RandomScale(0.2), 
                          RandomShear(0.2)])

        for name in combined_images: # 0-00005.jpg
            no, namenum = name.split('-')
            
            # './VOCdevkit_train/VOC2007/JPEGImages/' + 00005
            image_file = tf.io.read_file(os.path.join(IMAGE_DIR, namenum))
            
            image = tf.io.decode_jpeg(image_file, channels=3)
            
            bboxes = image2boxes.get(namenum)
            
            # 只對新加入的圖片進行
            if no != '0':
                image, bboxes = trans(image.numpy().copy(), bboxes.copy())
            
            # 將轉換後的image encode並寫入combined image directory (COM_IMAGE_DIR)
            image = tf.io.encode_jpeg(image)
            tf.io.write_file(os.path.join(COM_IMAGE_DIR, name), image)

            # 寫新的圖片檔名和 bounding box 進 combined data path (COM_DATA_PATH)
            new_record = name + ' '
            box_info = []
            for box in bboxes:
                for b in box:
                    box_info.append(str(b))
            new_record += ' '.join(box_info)
            new_record += '\n'
            f.write(new_record)

#             buff_list = [name] + [str(int(b)) for box in bboxes for b in box]
#             f.write(' '.join(buff_list) + '\n')

In [8]:
augment = True
if augment:
    gen_combined_dataset()

## Dataset Loader

In [9]:
class DatasetGenerator:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """

    def __init__(self):
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(COM_DATA_PATH, 'r')

        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])

            self.record_list.append([float(num) for num in ss[1:]])

            self.object_num_list.append(min(len(self.record_list[-1])//5, 
                                            MAX_OBJECTS_PER_IMAGE))
            if len(self.record_list[-1]) < MAX_OBJECTS_PER_IMAGE*5:
                # if there are objects less than MAX_OBJECTS_PER_IMAGE, pad the list
                self.record_list[-1] = self.record_list[-1] +\
                [0., 0., 0., 0., 0.]*\
                (MAX_OBJECTS_PER_IMAGE-len(self.record_list[-1])//5)
                
            elif len(self.record_list[-1]) > MAX_OBJECTS_PER_IMAGE*5:
               # if there are objects more than MAX_OBJECTS_PER_IMAGE, crop the list
                self.record_list[-1] = self.record_list[-1][:MAX_OBJECTS_PER_IMAGE*5]

    def _data_preprocess(self, image_name, raw_labels, object_num):
        image_file = tf.io.read_file(COM_IMAGE_DIR+image_name)
        image = tf.io.decode_jpeg(image_file, channels=3)

        h = tf.shape(image)[0]
        w = tf.shape(image)[1]

        width_ratio  = IMAGE_SIZE * 1.0 / tf.cast(w, tf.float32) 
        height_ratio = IMAGE_SIZE * 1.0 / tf.cast(h, tf.float32) 

        image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = (image/255) * 2 - 1

        raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)

        xmin = raw_labels[:, 0]
        ymin = raw_labels[:, 1]
        xmax = raw_labels[:, 2]
        ymax = raw_labels[:, 3]
        class_num = raw_labels[:, 4]

        xcenter = (xmin + xmax) * 1.0 / 2.0 * width_ratio
        ycenter = (ymin + ymax) * 1.0 / 2.0 * height_ratio

        box_w = (xmax - xmin) * width_ratio
        box_h = (ymax - ymin) * height_ratio

        labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis=1)

        return image, labels, tf.cast(object_num, tf.int32)

    def generate(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_names, 
                                                      np.array(self.record_list), 
                                                      np.array(self.object_num_list)))
        dataset = dataset.shuffle(100000)
        dataset = dataset.map(self._data_preprocess, 
                              num_parallel_calls = tf.data.experimental.AUTOTUNE)
        dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
        dataset = dataset.prefetch(buffer_size=200)

        return dataset

## Pretrained model + YOLO

https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [10]:
def conv_leaky_relu(inputs, filters, size, stride):
    x = layers.Conv2D(filters, size, stride, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.1)(x)

    return x

In [11]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

basemodel = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=img_inputs)
basemodel.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 223, 223, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 223, 223, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                               

                                                                                                  
 conv2d_10 (Conv2D)             (None, 53, 53, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 53, 53, 64)   12288       ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 53, 53, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 53, 53, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

 conv2d_21 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 batch_normalization_21 (BatchN  (None, 53, 53, 32)  96          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 53, 53, 48)   13824       ['activation_21[0][0]']          
          

 batch_normalization_26 (BatchN  (None, 53, 53, 32)  96          ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_29 (BatchN  (None, 53, 53, 64)  192         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_24 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activatio

 conv2d_37 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_4_ac[0][0]']           
                                                                                                  
 conv2d_40 (Conv2D)             (None, 53, 53, 48)   13824       ['activation_39[0][0]']          
                                                                                                  
 batch_normalization_37 (BatchN  (None, 53, 53, 32)  96          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 53, 53, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

...

 Normalization)                                                                                   
                                                                                                  
 activation_168 (Activation)    (None, 12, 12, 192)  0           ['batch_normalization_168[0][0]']
                                                                                                  
 conv2d_169 (Conv2D)            (None, 12, 12, 224)  129024      ['activation_168[0][0]']         
                                                                                                  
 batch_normalization_169 (Batch  (None, 12, 12, 224)  672        ['conv2d_169[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_169 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_169[0][0]']
          

                                                                                                  
 conv2d_177 (Conv2D)            (None, 12, 12, 224)  129024      ['activation_176[0][0]']         
                                                                                                  
 batch_normalization_177 (Batch  (None, 12, 12, 224)  672        ['conv2d_177[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_177 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_177[0][0]']
                                                                                                  
 conv2d_175 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_3_ac[0][0]']            
                                                                                                  
 conv2d_17

 batch_normalization_185 (Batch  (None, 12, 12, 224)  672        ['conv2d_185[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_185 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_185[0][0]']
                                                                                                  
 conv2d_183 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_5_ac[0][0]']            
                                                                                                  
 conv2d_186 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_185[0][0]']         
                                                                                                  
 batch_normalization_183 (Batch  (None, 12, 12, 192)  576        ['conv2d_183[0][0]']             
 Normaliza

 activation_193 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_193[0][0]']
                                                                                                  
 conv2d_191 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_7_ac[0][0]']            
                                                                                                  
 conv2d_194 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_193[0][0]']         
                                                                                                  
 batch_normalization_191 (Batch  (None, 12, 12, 192)  576        ['conv2d_191[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_194 (Batch  (None, 12, 12, 256)  768        ['conv2d_194[0][0]']             
 Normaliza

                                                                                                  
 conv2d_202 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_201[0][0]']         
                                                                                                  
 batch_normalization_199 (Batch  (None, 12, 12, 192)  576        ['conv2d_199[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_202 (Batch  (None, 12, 12, 256)  768        ['conv2d_202[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_199 (Activation)    (None, 12, 12, 192)  0           ['batch_normalization_199[0][0]']
          

In [13]:
for layer in basemodel.layers:
    layer.trainable = False
    
x = conv_leaky_relu(basemodel.output, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)

x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

final_model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='InceptionResnetV2_YOLO')
final_model.summary()

Model: "InceptionResnetV2_YOLO"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 223, 223, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 223, 223, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                            

                                                                                                  
 conv2d_10 (Conv2D)             (None, 53, 53, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 53, 53, 64)   12288       ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 53, 53, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 53, 53, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

 conv2d_21 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 batch_normalization_21 (BatchN  (None, 53, 53, 32)  96          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 53, 53, 48)   13824       ['activation_21[0][0]']          
          

 batch_normalization_26 (BatchN  (None, 53, 53, 32)  96          ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_29 (BatchN  (None, 53, 53, 64)  192         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_24 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 53, 53, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activatio

 conv2d_37 (Conv2D)             (None, 53, 53, 32)   10240       ['block35_4_ac[0][0]']           
                                                                                                  
 conv2d_40 (Conv2D)             (None, 53, 53, 48)   13824       ['activation_39[0][0]']          
                                                                                                  
 batch_normalization_37 (BatchN  (None, 53, 53, 32)  96          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 53, 53, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

...

 Normalization)                                                                                   
                                                                                                  
 activation_168 (Activation)    (None, 12, 12, 192)  0           ['batch_normalization_168[0][0]']
                                                                                                  
 conv2d_169 (Conv2D)            (None, 12, 12, 224)  129024      ['activation_168[0][0]']         
                                                                                                  
 batch_normalization_169 (Batch  (None, 12, 12, 224)  672        ['conv2d_169[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_169 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_169[0][0]']
          

                                                                                                  
 conv2d_177 (Conv2D)            (None, 12, 12, 224)  129024      ['activation_176[0][0]']         
                                                                                                  
 batch_normalization_177 (Batch  (None, 12, 12, 224)  672        ['conv2d_177[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_177 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_177[0][0]']
                                                                                                  
 conv2d_175 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_3_ac[0][0]']            
                                                                                                  
 conv2d_17

 batch_normalization_185 (Batch  (None, 12, 12, 224)  672        ['conv2d_185[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_185 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_185[0][0]']
                                                                                                  
 conv2d_183 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_5_ac[0][0]']            
                                                                                                  
 conv2d_186 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_185[0][0]']         
                                                                                                  
 batch_normalization_183 (Batch  (None, 12, 12, 192)  576        ['conv2d_183[0][0]']             
 Normaliza

 activation_193 (Activation)    (None, 12, 12, 224)  0           ['batch_normalization_193[0][0]']
                                                                                                  
 conv2d_191 (Conv2D)            (None, 12, 12, 192)  399360      ['block8_7_ac[0][0]']            
                                                                                                  
 conv2d_194 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_193[0][0]']         
                                                                                                  
 batch_normalization_191 (Batch  (None, 12, 12, 192)  576        ['conv2d_191[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_194 (Batch  (None, 12, 12, 256)  768        ['conv2d_194[0][0]']             
 Normaliza

                                                                                                  
 conv2d_202 (Conv2D)            (None, 12, 12, 256)  172032      ['activation_201[0][0]']         
                                                                                                  
 batch_normalization_199 (Batch  (None, 12, 12, 192)  576        ['conv2d_199[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_202 (Batch  (None, 12, 12, 256)  768        ['conv2d_202[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_199 (Activation)    (None, 12, 12, 192)  0           ['batch_normalization_199[0][0]']
          

## Define loss

In [14]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x,
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [15]:
def iou(boxes1, boxes2):
    """calculate ious
    Args:
      boxes1: 4-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4]  ====> (x_center, y_center, w, h)
      boxes2: 1-D tensor [4] ===> (x_center, y_center, w, h)

    Return:
      iou: 3-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
      ====> iou score for each cell
    """

    #boxes1 : [4(xmin, ymin, xmax, ymax), cell_size, cell_size, boxes_per_cell]
    boxes1 = tf.stack([boxes1[:, :, :, 0] - boxes1[:, :, :, 2] / 2, boxes1[:, :, :, 1] - boxes1[:, :, :, 3] / 2,
                      boxes1[:, :, :, 0] + boxes1[:, :, :, 2] / 2, boxes1[:, :, :, 1] + boxes1[:, :, :, 3] / 2])

    #boxes1 : [cell_size, cell_size, boxes_per_cell, 4(xmin, ymin, xmax, ymax)]
    boxes1 = tf.transpose(boxes1, [1, 2, 3, 0])

    boxes2 = tf.stack([boxes2[0] - boxes2[2] / 2, boxes2[1] - boxes2[3] / 2,
                       boxes2[0] + boxes2[2] / 2, boxes2[1] + boxes2[3] / 2])

    # calculate the left up point of boxes' overlap area
    lu = tf.maximum(boxes1[:, :, :, 0:2], boxes2[0:2])
    # calculate the right down point of boxes overlap area
    rd = tf.minimum(boxes1[:, :, :, 2:], boxes2[2:])

    # intersection
    intersection = rd - lu

    # the size of the intersection area
    inter_square = intersection[:, :, :, 0] * intersection[:, :, :, 1]

    mask = tf.cast(intersection[:, :, :, 0] > 0, tf.float32) * tf.cast(intersection[:, :, :, 1] > 0, tf.float32)

    # if intersection is negative, then the boxes don't overlap
    inter_square = mask * inter_square

    # calculate the boxs1 square and boxs2 square
    square1 = (boxes1[:, :, :, 2] - boxes1[:, :, :, 0]) * (boxes1[:, :, :, 3] - boxes1[:, :, :, 1])
    square2 = (boxes2[2] - boxes2[0]) * (boxes2[3] - boxes2[1])

    return inter_square/(square1 + square2 - inter_square + 1e-6)


def losses_calculation(predict, label):
    """
    calculate loss
    Args:
      predict: 3-D tensor [cell_size, cell_size, num_classes + 5 * boxes_per_cell]
      label :  [1, 5]  (x_center, y_center, w, h, class)
    """
    label = tf.reshape(label, [-1])

    # Step A. calculate objects tensor [CELL_SIZE, CELL_SIZE]
    # turn pixel position into cell position (corner)
    min_x = (label[0] - label[2] / 2) / (IMAGE_SIZE / CELL_SIZE)
    max_x = (label[0] + label[2] / 2) / (IMAGE_SIZE / CELL_SIZE)

    min_y = (label[1] - label[3] / 2) / (IMAGE_SIZE / CELL_SIZE)
    max_y = (label[1] + label[3] / 2) / (IMAGE_SIZE / CELL_SIZE)

    min_x = tf.floor(min_x)
    min_y = tf.floor(min_y)

    max_x = tf.minimum(tf.math.ceil(max_x), CELL_SIZE)
    max_y = tf.minimum(tf.math.ceil(max_y), CELL_SIZE)

    # calculate mask of object with cells
    onset = tf.cast(tf.stack([max_y - min_y, max_x - min_x]), dtype=tf.int32)
    object_mask = tf.ones(onset, tf.float32)

    offset = tf.cast(tf.stack([min_y, CELL_SIZE - max_y, min_x, CELL_SIZE - max_x]), tf.int32)
    offset = tf.reshape(offset, (2, 2))
    object_mask = tf.pad(object_mask, offset, "CONSTANT")

    # Step B. calculate the coordination of object center and the corresponding mask
    # turn pixel position into cell position (center)
    center_x = label[0] / (IMAGE_SIZE / CELL_SIZE)
    center_x = tf.floor(center_x)

    center_y = label[1] / (IMAGE_SIZE / CELL_SIZE)
    center_y = tf.floor(center_y)

    response = tf.ones([1, 1], tf.float32)

    # calculate the coordination of object center with cells
    objects_center_coord = tf.cast(tf.stack([center_y, CELL_SIZE - center_y - 1,
                                             center_x, CELL_SIZE - center_x - 1]),
                                   tf.int32)
    objects_center_coord = tf.reshape(objects_center_coord, (2, 2))

    # make mask
    objects_center_coord = tf.maximum(objects_center_coord, 0)   
    response = tf.pad(response, objects_center_coord, "CONSTANT")

    # Step C. calculate iou_predict_truth [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    predict_boxes = predict[:, :, NUM_CLASSES + BOXES_PER_CELL:]

    predict_boxes = tf.reshape(predict_boxes, [CELL_SIZE,
                                               CELL_SIZE,
                                               BOXES_PER_CELL, 4])
    # cell position to pixel position
    predict_boxes = predict_boxes * [IMAGE_SIZE / CELL_SIZE,
                                     IMAGE_SIZE / CELL_SIZE,
                                     IMAGE_SIZE, IMAGE_SIZE]

    # if there's no predict_box in that cell, then the base_boxes will be calcuated with label and got iou equals 0
    predict_boxes = base_boxes + predict_boxes

    iou_predict_truth = iou(predict_boxes, label[0:4])

    # calculate C tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    C = iou_predict_truth * tf.reshape(response, [CELL_SIZE, CELL_SIZE, 1])

    # calculate I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    I = iou_predict_truth * tf.reshape(response, [CELL_SIZE, CELL_SIZE, 1])

    max_I = tf.reduce_max(I, 2, keepdims=True)

    # replace large iou scores with response (object center) value
    I = tf.cast((I >= max_I), tf.float32) * tf.reshape(response, (CELL_SIZE, CELL_SIZE, 1))

    # calculate no_I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    no_I = tf.ones_like(I, dtype=tf.float32) - I

    p_C = predict[:, :, NUM_CLASSES:NUM_CLASSES + BOXES_PER_CELL]

    # calculate truth x, y, sqrt_w, sqrt_h 0-D
    x = label[0]
    y = label[1]

    sqrt_w = tf.sqrt(tf.abs(label[2]))
    sqrt_h = tf.sqrt(tf.abs(label[3]))

    # calculate predict p_x, p_y, p_sqrt_w, p_sqrt_h 3-D [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    p_x = predict_boxes[:, :, :, 0]
    p_y = predict_boxes[:, :, :, 1]

    p_sqrt_w = tf.sqrt(tf.minimum(IMAGE_SIZE * 1.0, tf.maximum(0.0, predict_boxes[:, :, :, 2])))
    p_sqrt_h = tf.sqrt(tf.minimum(IMAGE_SIZE * 1.0, tf.maximum(0.0, predict_boxes[:, :, :, 3])))

    # calculate ground truth p 1-D tensor [NUM_CLASSES]
    P = tf.one_hot(tf.cast(label[4], tf.int32), NUM_CLASSES, dtype=tf.float32)

    # calculate predicted p_P 3-D tensor [CELL_SIZE, CELL_SIZE, NUM_CLASSES]
    p_P = predict[:, :, 0:NUM_CLASSES]

    # class_loss
    class_loss = tf.nn.l2_loss(tf.reshape(object_mask, (CELL_SIZE, CELL_SIZE, 1)) * (p_P - P)) * CLASS_SCALE

    # object_loss
    object_loss = tf.nn.l2_loss(I * (p_C - C)) * OBJECT_SCALE

    # noobject_loss
    noobject_loss = tf.nn.l2_loss(no_I * (p_C)) * NOOBJECT_SCALE

    # coord_loss
    coord_loss = (tf.nn.l2_loss(I * (p_x - x) / (IMAGE_SIZE / CELL_SIZE)) +
                  tf.nn.l2_loss(I * (p_y - y) / (IMAGE_SIZE / CELL_SIZE)) +
                  tf.nn.l2_loss(I * (p_sqrt_w - sqrt_w)) / IMAGE_SIZE +
                  tf.nn.l2_loss(I * (p_sqrt_h - sqrt_h)) / IMAGE_SIZE) * COORD_SCALE

    return class_loss + object_loss + noobject_loss + coord_loss

In [16]:
def yolo_loss(predicts, labels, objects_num):
    """
    Add Loss to all the trainable variables
    Args:
        predicts:    4-D tensor [batch_size, cell_size, cell_size, num_classes + 5 * boxes_per_cell]
                     ===> (num_classes, boxes_per_cell, 4 * boxes_per_cell)
        labels:      3-D tensor of [batch_size, max_objects, 5]
        objects_num: 1-D tensor [batch_size]
    """

    loss = 0.
    # you can parallel the code with tf.map_fn or tf.vectorized_map (big performance gain!)
    for i in tf.range(BATCH_SIZE):
        predict = predicts[i, :, :, :]
        label = labels[i, :, :]
        object_num = objects_num[i]

        for j in tf.range(object_num):
            loss = loss + losses_calculation(predict, label[j:j+1, :])

    return loss / BATCH_SIZE

## Start Training

In [ ]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=final_model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/InceptionResnetV2_YOLO', max_to_keep=3,
                                     checkpoint_name='InceptionResnetV2_YOLO')

In [ ]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = final_model(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2 * 4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, final_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, final_model.trainable_weights))

In [ ]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(tqdm(dataset)):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))

## Predict Test data
### Process YOLO's predictions

In [ ]:
def process_outputs(outputs):
    """
    Process YOLO outputs into bounding boxes
    """

    class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
    conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
    class_probs = np.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
    confs = np.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
    boxes = np.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
    predicts = np.concatenate([class_probs, confs, boxes], 3)

    p_classes = predicts[0, :, :, 0:20]
    C = predicts[0, :, :, 20:22]
    coordinate = predicts[0, :, :, 22:]

    p_classes = np.reshape(p_classes, (CELL_SIZE, CELL_SIZE, 1, 20))
    C = np.reshape(C, (CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 1))

    P = C * p_classes
    #P's shape [7, 7, 2, 20]

    #choose the most confidence one
    max_conf = np.max(P)
    index = np.argmax(P)

    index = np.unravel_index(index, P.shape)

    class_num = index[3]

    coordinate = np.reshape(coordinate, 
                            (CELL_SIZE, 
                             CELL_SIZE,
                             BOXES_PER_CELL, 
                             4))

    max_coordinate = coordinate[index[0], index[1], index[2], :]

    xcenter = max_coordinate[0]
    ycenter = max_coordinate[1]
    w = max_coordinate[2]
    h = max_coordinate[3]

    xcenter = (index[1] + xcenter) * (IMAGE_SIZE/float(CELL_SIZE))
    ycenter = (index[0] + ycenter) * (IMAGE_SIZE/float(CELL_SIZE))

    w = w * IMAGE_SIZE
    h = h * IMAGE_SIZE

    xmin = xcenter - w/2.0
    ymin = ycenter - h/2.0

    xmax = xmin + w
    ymax = ymin + h

    return xmin, ymin, xmax, ymax, class_num, max_conf

### Build Test dataset Iterator

In [ ]:
test_img_files = open('./pascal_voc_testing_data.txt')
test_img_dir = './VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []

for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])

test_dataset = tf.data.Dataset.from_tensor_slices(test_images)

def load_img_data(image_name):
    image_file = tf.io.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)

    h = tf.shape(image)[0]
    w = tf.shape(image)[1]

    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = (image/255) * 2 - 1

    return image_name, image, h, w

test_dataset = test_dataset.map(load_img_data, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [ ]:
ckpt = tf.train.Checkpoint(net=final_model)
ckpt.restore('./ckpts/InceptionResnetV2_YOLO/InceptionResnetV2_YOLO-40')

### Make Prediction and Output to txt file

In [ ]:
@tf.function
def prediction_step(img):
    return final_model(img, training=False)

In [ ]:
output_file = open('./test_prediction.txt', 'w')

for img_name, test_img, img_h, img_w in test_dataset:
    batch_num = img_name.shape[0]
    for i in range(batch_num):
        xmin, ymin, xmax, ymax, class_num, conf = process_outputs(prediction_step(test_img[i:i+1]))
        xmin, ymin, xmax, ymax = xmin*(img_w[i:i+1]/IMAGE_SIZE), ymin*(img_h[i:i+1]/IMAGE_SIZE), xmax*(img_w[i:i+1]/IMAGE_SIZE), ymax*(img_h[i:i+1]/IMAGE_SIZE)

        #img filename, xmin, ymin, xmax, ymax, class, confidence
        output_file.write(img_name[i:i+1].numpy()[0].decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))

output_file.close()

### Run Evaluation Metric

In [ ]:
import sys
sys.path.insert(0, './evaluate')

import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./test_prediction.txt', './output_file_InceptionResnetV2_YOLO.csv')

import pandas as pd
cap = pd.read_csv('./output_file_InceptionResnetV2_YOLO.csv')['packedCAP']
print('score: {:f}'.format(sum((1. - cap) ** 2) / len(cap)))

## Visualization

In [ ]:
np_img = cv2.imread('./VOCdevkit_test/VOC2007/JPEGImages/000002.jpg')
resized_img = cv2.resize(np_img, (IMAGE_SIZE, IMAGE_SIZE))
np_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
resized_img = np_img
np_img = np_img.astype(np.float32)
# np_img = tf.keras.applications.densenet.preprocess_input(np_img)

np_img = np_img / 255.0 * 2 - 1
np_img = np.reshape(np_img, (1, IMAGE_SIZE, IMAGE_SIZE, 3))


y_pred = final_model(np_img, training=False)
xmin, ymin, xmax, ymax, class_num, conf = process_outputs(y_pred)
class_name = classes_name[class_num]
cv2.rectangle(resized_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 255), 3)
cv2.putText(resized_img, class_name, (0, 200), 2, 1.5, (0, 255, 255), 2)

plt.imshow(resized_img)
plt.show()

## Xception + YOLO

In [46]:
class DatasetGenerator:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """

    def __init__(self):
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(COM_DATA_PATH, 'r')

        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])

            self.record_list.append([float(num) for num in ss[1:]])

            self.object_num_list.append(
                min(len(self.record_list[-1]) // 5, MAX_OBJECTS_PER_IMAGE))
            if len(self.record_list[-1]) < MAX_OBJECTS_PER_IMAGE * 5:
                # if there are objects less than MAX_OBJECTS_PER_IMAGE, pad the list
                self.record_list[-1] = self.record_list[-1] + \
                    [0., 0., 0., 0., 0.] * \
                    (MAX_OBJECTS_PER_IMAGE - len(self.record_list[-1]) // 5)

            elif len(self.record_list[-1]) > MAX_OBJECTS_PER_IMAGE * 5:
               # if there are objects more than MAX_OBJECTS_PER_IMAGE, crop the list
                self.record_list[-1] = self.record_list[-1][:MAX_OBJECTS_PER_IMAGE * 5]

    def _data_preprocess(self, image_name, raw_labels, object_num):
        image_file = tf.io.read_file(COM_IMAGE_DIR + image_name)
        image = tf.io.decode_jpeg(image_file, channels=3)

        h = tf.shape(image)[0]
        w = tf.shape(image)[1]

        width_ratio = IMAGE_SIZE * 1.0 / tf.cast(w, tf.float32)
        height_ratio = IMAGE_SIZE * 1.0 / tf.cast(h, tf.float32)

        image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = (image/255) * 2 - 1
#         image = tf.keras.applications.xception.preprocess_input(image)

        raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)

        xmin = raw_labels[:, 0]
        ymin = raw_labels[:, 1]
        xmax = raw_labels[:, 2]
        ymax = raw_labels[:, 3]
        class_num = raw_labels[:, 4]

        xcenter = (xmin + xmax) * 1.0 / 2.0 * width_ratio
        ycenter = (ymin + ymax) * 1.0 / 2.0 * height_ratio

        box_w = (xmax - xmin) * width_ratio
        box_h = (ymax - ymin) * height_ratio

        labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis=1)

        return image, labels, tf.cast(object_num, tf.int32)

    def generate(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_names,
                                                      np.array(self.record_list),
                                                      np.array(self.object_num_list)))
        dataset = dataset.shuffle(100000)\
                         .map(self._data_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                         .batch(BATCH_SIZE, drop_remainder=True)\
                         .prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

In [47]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

basemodel = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=img_inputs)
basemodel.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 223, 223, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 223, 223, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

 2D)                                                                                              
                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, 56, 56, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 56, 56, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 56, 56, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
          

                                                                                                  
 block7_sepconv1_act (Activatio  (None, 28, 28, 728)  0          ['add_4[0][0]']                  
 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 28, 28, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_se

 2D)                                                                                              
                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 28, 28, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 28, 28, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
          

 block12_sepconv2_bn (BatchNorm  (None, 28, 28, 728)  2912       ['block12_sepconv2[0][0]']       
 alization)                                                                                       
                                                                                                  
 block12_sepconv3_act (Activati  (None, 28, 28, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 28, 28, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 28, 28, 728)  2912       ['block12_sepconv3[0][0]']       
 alization

In [48]:
basemodel.trainable = False
x = conv_leaky_relu(basemodel.output, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)

x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

final_model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='Xception_YOLO')
final_model.summary()

Model: "Xception_YOLO"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 223, 223, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 223, 223, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                     

 2D)                                                                                              
                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, 56, 56, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 56, 56, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 56, 56, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
          

                                                                                                  
 block7_sepconv1_act (Activatio  (None, 28, 28, 728)  0          ['add_4[0][0]']                  
 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 28, 28, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_se

 2D)                                                                                              
                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 28, 28, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 28, 28, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
          

 block12_sepconv2_bn (BatchNorm  (None, 28, 28, 728)  2912       ['block12_sepconv2[0][0]']       
 alization)                                                                                       
                                                                                                  
 block12_sepconv3_act (Activati  (None, 28, 28, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 28, 28, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 28, 28, 728)  2912       ['block12_sepconv3[0][0]']       
 alization

 batch_normalization_219 (Batch  (None, 7, 7, 1024)  4096        ['conv2d_219[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu_14 (LeakyReLU)     (None, 7, 7, 1024)   0           ['batch_normalization_219[0][0]']
                                                                                                  
 conv2d_220 (Conv2D)            (None, 7, 7, 1024)   9438208     ['leaky_re_lu_14[0][0]']         
                                                                                                  
 batch_normalization_220 (Batch  (None, 7, 7, 1024)  4096        ['conv2d_220[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 leaky_re_

In [49]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x,
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [50]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=final_model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/Xception_YOLO', max_to_keep=3,
                                     checkpoint_name='Xception_YOLO')

In [51]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = final_model(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, final_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, final_model.trainable_weights))

In [52]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(tqdm(dataset)):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))

2023-11-21 06:48:15.632983, start training.


100%|██████████| 5482/5482 [06:30<00:00, 14.06it/s]


2023-11-21 06:54:45.663912, Epoch 1: loss 5.64
Saved checkpoint for epoch 1: ./ckpts/Xception_YOLO\Xception_YOLO-1


100%|██████████| 5482/5482 [06:27<00:00, 14.13it/s]


2023-11-21 07:01:15.255137, Epoch 2: loss 3.25
Saved checkpoint for epoch 2: ./ckpts/Xception_YOLO\Xception_YOLO-2


...

2023-11-21 10:54:46.949593, Epoch 38: loss 1.27
Saved checkpoint for epoch 38: ./ckpts/Xception_YOLO\Xception_YOLO-38


100%|██████████| 5482/5482 [06:27<00:00, 14.15it/s]


2023-11-21 11:01:16.244541, Epoch 39: loss 1.26
Saved checkpoint for epoch 39: ./ckpts/Xception_YOLO\Xception_YOLO-39


100%|██████████| 5482/5482 [06:27<00:00, 14.14it/s]


2023-11-21 11:07:45.698174, Epoch 40: loss 1.25
Saved checkpoint for epoch 40: ./ckpts/Xception_YOLO\Xception_YOLO-40


In [53]:
test_img_files = open('./pascal_voc_testing_data.txt')
test_img_dir = './VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []

for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])

test_dataset = tf.data.Dataset.from_tensor_slices(test_images)

def load_img_data(image_name):
    image_file = tf.io.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)

    h = tf.shape(image)[0]
    w = tf.shape(image)[1]

    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = (image/255) * 2 - 1

    return image_name, image, h, w

test_dataset = test_dataset.map(load_img_data, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [54]:
ckpt = tf.train.Checkpoint(net=final_model)
ckpt.restore('./ckpts/Xception_YOLO/Xception_YOLO-40')

In [55]:
output_file = open('./test_prediction.txt', 'w')

for img_name, test_img, img_h, img_w in test_dataset:
    batch_num = img_name.shape[0]
    for i in range(batch_num):
        xmin, ymin, xmax, ymax, class_num, conf = process_outputs(prediction_step(test_img[i:i+1]))
        xmin, ymin, xmax, ymax = xmin*(img_w[i:i+1]/IMAGE_SIZE), ymin*(img_h[i:i+1]/IMAGE_SIZE), xmax*(img_w[i:i+1]/IMAGE_SIZE), ymax*(img_h[i:i+1]/IMAGE_SIZE)

        #img filename, xmin, ymin, xmax, ymax, class, confidence
        output_file.write(img_name[i:i+1].numpy()[0].decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))

output_file.close()

In [65]:
import sys
sys.path.insert(0, './evaluate')

import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./test_prediction.txt', './output_file_Xception.csv')

import pandas as pd
cap = pd.read_csv('./output_file_Xception.csv')['packedCAP']
print('score: {:f}'.format(sum((1. - cap) ** 2) / len(cap)))

End Evalutation
score: 0.471578


## ResNet50V2 + YOLO

In [57]:
class DatasetGenerator:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """

    def __init__(self):
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(COM_DATA_PATH, 'r')

        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])

            self.record_list.append([float(num) for num in ss[1:]])

            self.object_num_list.append(
                min(len(self.record_list[-1]) // 5, MAX_OBJECTS_PER_IMAGE))
            if len(self.record_list[-1]) < MAX_OBJECTS_PER_IMAGE * 5:
                # if there are objects less than MAX_OBJECTS_PER_IMAGE, pad the list
                self.record_list[-1] = self.record_list[-1] + \
                    [0., 0., 0., 0., 0.] * \
                    (MAX_OBJECTS_PER_IMAGE - len(self.record_list[-1]) // 5)

            elif len(self.record_list[-1]) > MAX_OBJECTS_PER_IMAGE * 5:
               # if there are objects more than MAX_OBJECTS_PER_IMAGE, crop the list
                self.record_list[-1] = self.record_list[-1][:MAX_OBJECTS_PER_IMAGE * 5]

    def _data_preprocess(self, image_name, raw_labels, object_num):
        image_file = tf.io.read_file(COM_IMAGE_DIR + image_name)
        image = tf.io.decode_jpeg(image_file, channels=3)

        h = tf.shape(image)[0]
        w = tf.shape(image)[1]

        width_ratio = IMAGE_SIZE * 1.0 / tf.cast(w, tf.float32)
        height_ratio = IMAGE_SIZE * 1.0 / tf.cast(h, tf.float32)

        image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
        # image = (image/255) * 2 - 1
        image = tf.keras.applications.xception.preprocess_input(image)

        raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)

        xmin = raw_labels[:, 0]
        ymin = raw_labels[:, 1]
        xmax = raw_labels[:, 2]
        ymax = raw_labels[:, 3]
        class_num = raw_labels[:, 4]

        xcenter = (xmin + xmax) * 1.0 / 2.0 * width_ratio
        ycenter = (ymin + ymax) * 1.0 / 2.0 * height_ratio

        box_w = (xmax - xmin) * width_ratio
        box_h = (ymax - ymin) * height_ratio

        labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis=1)

        return image, labels, tf.cast(object_num, tf.int32)

    def generate(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_names,
                                                      np.array(self.record_list),
                                                      np.array(self.object_num_list)))
        dataset = dataset.shuffle(100000)\
                         .map(self._data_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                         .batch(BATCH_SIZE, drop_remainder=True)\
                         .prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

In [58]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
basemodel = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights='imagenet', input_tensor=img_inputs)
basemodel.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 454, 454, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 224, 224, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

 conv2_block2_out (Add)         (None, 112, 112, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 112, 112, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 112, 112, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 112, 112, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 58, 58, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 56, 56, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_blo

 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 28, 28, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 28, 28, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block3_2_conv (Conv2D)   (None, 28, 28, 256)  589824      ['conv4_block3_2_pad[0][0]']     
                                                                                                  
 conv4_block3_2_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 28, 28, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 28, 28, 1024  263168      ['conv4_block3_2_relu[0][0]']    
          

 vation)                        )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 28, 28, 256)  262144      ['conv4_block6_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 28, 28, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv5_block2_2_conv (Conv2D)   (None, 14, 14, 512)  2359296     ['conv5_block2_2_pad[0][0]']     
                                                                                                  
 conv5_block2_2_bn (BatchNormal  (None, 14, 14, 512)  2048       ['conv5_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_2_relu (Activatio  (None, 14, 14, 512)  0          ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 14, 14, 2048  1050624     ['conv5_block2_2_relu[0][0]']    
          

In [59]:
basemodel.trainable = False
x = conv_leaky_relu(basemodel.output, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)

x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

final_model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='ResNet50V2_YOLO')
final_model.summary()

Model: "ResNet50V2_YOLO"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 454, 454, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 224, 224, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

 conv2_block2_out (Add)         (None, 112, 112, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 112, 112, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 112, 112, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 112, 112, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 58, 58, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 56, 56, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_blo

 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 28, 28, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 28, 28, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block3_2_conv (Conv2D)   (None, 28, 28, 256)  589824      ['conv4_block3_2_pad[0][0]']     
                                                                                                  
 conv4_block3_2_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 28, 28, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 28, 28, 1024  263168      ['conv4_block3_2_relu[0][0]']    
          

 vation)                        )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 28, 28, 256)  262144      ['conv4_block6_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 28, 28, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv5_block2_2_conv (Conv2D)   (None, 14, 14, 512)  2359296     ['conv5_block2_2_pad[0][0]']     
                                                                                                  
 conv5_block2_2_bn (BatchNormal  (None, 14, 14, 512)  2048       ['conv5_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_2_relu (Activatio  (None, 14, 14, 512)  0          ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 14, 14, 2048  1050624     ['conv5_block2_2_relu[0][0]']    
          

 leaky_re_lu_21 (LeakyReLU)     (None, 7, 7, 1024)   0           ['batch_normalization_225[0][0]']
                                                                                                  
 conv2d_226 (Conv2D)            (None, 7, 7, 1024)   9438208     ['leaky_re_lu_21[0][0]']         
                                                                                                  
 batch_normalization_226 (Batch  (None, 7, 7, 1024)  4096        ['conv2d_226[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu_22 (LeakyReLU)     (None, 7, 7, 1024)   0           ['batch_normalization_226[0][0]']
                                                                                                  
 flatten_3 (Flatten)            (None, 50176)        0           ['leaky_re_lu_22[0][0]']         
          

In [60]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x,
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [61]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=final_model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/ResNet50V2_YOLO', max_to_keep=3,
                                     checkpoint_name='ResNet50V2_YOLO')

In [62]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = final_model(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, final_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, final_model.trainable_weights))

In [68]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(tqdm(dataset)):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))

2023-11-21 11:35:43.905197, start training.


100%|██████████| 5482/5482 [06:05<00:00, 14.99it/s]


2023-11-21 11:41:49.653276, Epoch 1: loss 3.25
Saved checkpoint for epoch 3: ./ckpts/ResNet50V2_YOLO\ResNet50V2_YOLO-2


100%|██████████| 5482/5482 [06:28<00:00, 14.10it/s]


2023-11-21 11:48:20.310026, Epoch 2: loss 2.67
Saved checkpoint for epoch 4: ./ckpts/ResNet50V2_YOLO\ResNet50V2_YOLO-3


...

2023-11-21 15:34:26.698059, Epoch 38: loss 1.25
Saved checkpoint for epoch 40: ./ckpts/ResNet50V2_YOLO\ResNet50V2_YOLO-39


100%|██████████| 5482/5482 [06:03<00:00, 15.08it/s]


2023-11-21 15:40:32.016432, Epoch 39: loss 1.24
Saved checkpoint for epoch 41: ./ckpts/ResNet50V2_YOLO\ResNet50V2_YOLO-40


100%|██████████| 5482/5482 [06:03<00:00, 15.09it/s]


2023-11-21 15:46:37.122609, Epoch 40: loss 1.24
Saved checkpoint for epoch 42: ./ckpts/ResNet50V2_YOLO\ResNet50V2_YOLO-41


In [69]:
test_img_files = open('./pascal_voc_testing_data.txt')
test_img_dir = './VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []

for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])

test_dataset = tf.data.Dataset.from_tensor_slices(test_images)

def load_img_data(image_name):
    image_file = tf.io.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)

    h = tf.shape(image)[0]
    w = tf.shape(image)[1]

    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = (image/255) * 2 - 1

    return image_name, image, h, w

test_dataset = test_dataset.map(load_img_data, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [70]:
ckpt = tf.train.Checkpoint(net=final_model)
ckpt.restore('./ckpts/ResNet50V2_YOLO/ResNet50V2_YOLO-40')

In [71]:
output_file = open('./test_prediction.txt', 'w')

for img_name, test_img, img_h, img_w in test_dataset:
    batch_num = img_name.shape[0]
    for i in range(batch_num):
        xmin, ymin, xmax, ymax, class_num, conf = process_outputs(prediction_step(test_img[i:i+1]))
        xmin, ymin, xmax, ymax = xmin*(img_w[i:i+1]/IMAGE_SIZE), ymin*(img_h[i:i+1]/IMAGE_SIZE), xmax*(img_w[i:i+1]/IMAGE_SIZE), ymax*(img_h[i:i+1]/IMAGE_SIZE)

        #img filename, xmin, ymin, xmax, ymax, class, confidence
        output_file.write(img_name[i:i+1].numpy()[0].decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))

output_file.close()

In [72]:
import sys
sys.path.insert(0, './evaluate')

import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./test_prediction.txt', './output_file_ResNet50V2.csv')

cap = pd.read_csv('./output_file_ResNet50V2.csv')['packedCAP']
print('score: {:f}'.format(sum((1. - cap) ** 2) / len(cap)))

End Evalutation
score: 0.471578


## NASNetLarge + YOLO

In [9]:
class DatasetGenerator:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """

    def __init__(self):
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(COM_DATA_PATH, 'r')

        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])

            self.record_list.append([float(num) for num in ss[1:]])

            self.object_num_list.append(
                min(len(self.record_list[-1]) // 5, MAX_OBJECTS_PER_IMAGE))
            if len(self.record_list[-1]) < MAX_OBJECTS_PER_IMAGE * 5:
                # if there are objects less than MAX_OBJECTS_PER_IMAGE, pad the list
                self.record_list[-1] = self.record_list[-1] + \
                    [0., 0., 0., 0., 0.] * \
                    (MAX_OBJECTS_PER_IMAGE - len(self.record_list[-1]) // 5)

            elif len(self.record_list[-1]) > MAX_OBJECTS_PER_IMAGE * 5:
               # if there are objects more than MAX_OBJECTS_PER_IMAGE, crop the list
                self.record_list[-1] = self.record_list[-1][:MAX_OBJECTS_PER_IMAGE * 5]

    def _data_preprocess(self, image_name, raw_labels, object_num):
        image_file = tf.io.read_file(COM_IMAGE_DIR + image_name)
        image = tf.io.decode_jpeg(image_file, channels=3)

        h = tf.shape(image)[0]
        w = tf.shape(image)[1]

        width_ratio = IMAGE_SIZE * 1.0 / tf.cast(w, tf.float32)
        height_ratio = IMAGE_SIZE * 1.0 / tf.cast(h, tf.float32)

        image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
        # image = (image/255) * 2 - 1
        image = tf.keras.applications.nasnet.preprocess_input(image)

        raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)

        xmin = raw_labels[:, 0]
        ymin = raw_labels[:, 1]
        xmax = raw_labels[:, 2]
        ymax = raw_labels[:, 3]
        class_num = raw_labels[:, 4]

        xcenter = (xmin + xmax) * 1.0 / 2.0 * width_ratio
        ycenter = (ymin + ymax) * 1.0 / 2.0 * height_ratio

        box_w = (xmax - xmin) * width_ratio
        box_h = (ymax - ymin) * height_ratio

        labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis=1)

        return image, labels, tf.cast(object_num, tf.int32)

    def generate(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_names,
                                                      np.array(self.record_list),
                                                      np.array(self.object_num_list)))
        dataset = dataset.shuffle(100000)\
                         .map(self._data_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                         .batch(BATCH_SIZE, drop_remainder=True)\
                         .prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

In [10]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
basemodel = tf.keras.applications.nasnet.NASNetLarge(include_top=False, weights='imagenet', input_tensor=img_inputs)
basemodel.summary()

Model: "NASNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 223, 223, 96  2592        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 223, 223, 96  384         ['stem_conv1[0][0]']             
                                )                                                            

                                                                                                  
 separable_conv_1_pad_reduction  (None, 227, 227, 96  0          ['activation_7[0][0]']           
 _right3_stem_1 (ZeroPadding2D)  )                                                                
                                                                                                  
 activation_9 (Activation)      (None, 112, 112, 42  0           ['reduction_add_1_stem_1[0][0]'] 
                                )                                                                 
                                                                                                  
 separable_conv_1_bn_reduction_  (None, 112, 112, 42  168        ['separable_conv_1_reduction_righ
 right2_stem_1 (BatchNormalizat  )                               t2_stem_1[0][0]']                
 ion)                                                                                             
          

                                                                                                  
 add (Add)                      (None, 112, 112, 42  0           ['reduction_add_2_stem_1[0][0]', 
                                )                                 'reduction_left4_stem_1[0][0]'] 
                                                                                                  
 reduction_add4_stem_1 (Add)    (None, 112, 112, 42  0           ['separable_conv_2_bn_reduction_l
                                )                                eft4_stem_1[0][0]',              
                                                                  'reduction_right5_stem_1[0][0]']
                                                                                                  
 cropping2d (Cropping2D)        (None, 223, 223, 96  0           ['zero_padding2d[0][0]']         
                                )                                                                 
          

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_left
 left1_stem_2 (BatchNormalizati                                  1_stem_2[0][0]']                 
 on)                                                                                              
                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_righ
 right1_stem_2 (BatchNormalizat                                  t1_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable_conv_1_pad_reduction  (None, 117, 117, 84  0          ['activation_16[0][0]']          
 _right2_s

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_righ
 right3_stem_2 (BatchNormalizat                                  t3_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 reduction_left4_stem_2 (Averag  (None, 56, 56, 84)  0           ['reduction_add_1_stem_2[0][0]'] 
 ePooling2D)                                                                                      
                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_left
 left4_stem_2 (BatchNormalizati                                  4_stem_2[0][0]']                 
 on)      

...

 separable_conv_1_bn_normal_rig  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_right1_
 ht1_15 (BatchNormalization)                                     15[0][0]']                       
                                                                                                  
 separable_conv_1_bn_normal_lef  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_left2_1
 t2_15 (BatchNormalization)                                      5[0][0]']                        
                                                                                                  
 separable_conv_1_bn_normal_rig  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_right2_
 ht2_15 (BatchNormalization)                                     15[0][0]']                       
                                                                                                  
 separable_conv_1_bn_normal_lef  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_left5_1
 t5_15 (Ba

                                                                 5_15[0][0]',                     
                                                                  'normal_bn_1_15[0][0]']         
                                                                                                  
 normal_concat_15 (Concatenate)  (None, 14, 14, 4032  0          ['adjust_bn_15[0][0]',           
                                )                                 'normal_add_1_15[0][0]',        
                                                                  'normal_add_2_15[0][0]',        
                                                                  'normal_add_3_15[0][0]',        
                                                                  'normal_add_4_15[0][0]',        
                                                                  'normal_add_5_15[0][0]']        
                                                                                                  
 activatio

 16 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_2_normal_right1  (None, 14, 14, 672)  457632     ['activation_228[0][0]']         
 _16 (SeparableConv2D)                                                                            
                                                                                                  
 separable_conv_2_normal_left2_  (None, 14, 14, 672)  468384     ['activation_230[0][0]']         
 16 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_2_normal_right2  (None, 14, 14, 672)  457632     ['activation_232[0][0]']         
 _16 (SeparableConv2D)                                                                            
          

                                                                                                  
 activation_239 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_241 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_243 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_245 (Activation)    (None, 14, 14, 672)  0           ['normal_bn_1_17[0][0]']         
                                                                                                  
 separable_conv_1_normal_left1_  (None, 14, 14, 672)  468384     ['activation_237[0][0]']         
 17 (Separ

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
basemodel.trainable = False
x = conv_leaky_relu(basemodel.output, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)

x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

final_model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='NASNetLarge_YOLO')
final_model.summary()

Model: "NASNetLarge_YOLO"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 223, 223, 96  2592        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 223, 223, 96  384         ['stem_conv1[0][0]']             
                                )                                                  

                                                                                                  
 separable_conv_1_pad_reduction  (None, 227, 227, 96  0          ['activation_7[0][0]']           
 _right3_stem_1 (ZeroPadding2D)  )                                                                
                                                                                                  
 activation_9 (Activation)      (None, 112, 112, 42  0           ['reduction_add_1_stem_1[0][0]'] 
                                )                                                                 
                                                                                                  
 separable_conv_1_bn_reduction_  (None, 112, 112, 42  168        ['separable_conv_1_reduction_righ
 right2_stem_1 (BatchNormalizat  )                               t2_stem_1[0][0]']                
 ion)                                                                                             
          

                                                                                                  
 add (Add)                      (None, 112, 112, 42  0           ['reduction_add_2_stem_1[0][0]', 
                                )                                 'reduction_left4_stem_1[0][0]'] 
                                                                                                  
 reduction_add4_stem_1 (Add)    (None, 112, 112, 42  0           ['separable_conv_2_bn_reduction_l
                                )                                eft4_stem_1[0][0]',              
                                                                  'reduction_right5_stem_1[0][0]']
                                                                                                  
 cropping2d (Cropping2D)        (None, 223, 223, 96  0           ['zero_padding2d[0][0]']         
                                )                                                                 
          

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_left
 left1_stem_2 (BatchNormalizati                                  1_stem_2[0][0]']                 
 on)                                                                                              
                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_righ
 right1_stem_2 (BatchNormalizat                                  t1_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable_conv_1_pad_reduction  (None, 117, 117, 84  0          ['activation_16[0][0]']          
 _right2_s

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_righ
 right3_stem_2 (BatchNormalizat                                  t3_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 reduction_left4_stem_2 (Averag  (None, 56, 56, 84)  0           ['reduction_add_1_stem_2[0][0]'] 
 ePooling2D)                                                                                      
                                                                                                  
 separable_conv_2_bn_reduction_  (None, 56, 56, 84)  336         ['separable_conv_2_reduction_left
 left4_stem_2 (BatchNormalizati                                  4_stem_2[0][0]']                 
 on)      

...

 separable_conv_1_bn_normal_rig  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_right1_
 ht1_15 (BatchNormalization)                                     15[0][0]']                       
                                                                                                  
 separable_conv_1_bn_normal_lef  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_left2_1
 t2_15 (BatchNormalization)                                      5[0][0]']                        
                                                                                                  
 separable_conv_1_bn_normal_rig  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_right2_
 ht2_15 (BatchNormalization)                                     15[0][0]']                       
                                                                                                  
 separable_conv_1_bn_normal_lef  (None, 14, 14, 672)  2688       ['separable_conv_1_normal_left5_1
 t5_15 (Ba

                                                                 5_15[0][0]',                     
                                                                  'normal_bn_1_15[0][0]']         
                                                                                                  
 normal_concat_15 (Concatenate)  (None, 14, 14, 4032  0          ['adjust_bn_15[0][0]',           
                                )                                 'normal_add_1_15[0][0]',        
                                                                  'normal_add_2_15[0][0]',        
                                                                  'normal_add_3_15[0][0]',        
                                                                  'normal_add_4_15[0][0]',        
                                                                  'normal_add_5_15[0][0]']        
                                                                                                  
 activatio

 16 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_2_normal_right1  (None, 14, 14, 672)  457632     ['activation_228[0][0]']         
 _16 (SeparableConv2D)                                                                            
                                                                                                  
 separable_conv_2_normal_left2_  (None, 14, 14, 672)  468384     ['activation_230[0][0]']         
 16 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_2_normal_right2  (None, 14, 14, 672)  457632     ['activation_232[0][0]']         
 _16 (SeparableConv2D)                                                                            
          

                                                                                                  
 activation_239 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_241 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_243 (Activation)    (None, 14, 14, 672)  0           ['adjust_bn_17[0][0]']           
                                                                                                  
 activation_245 (Activation)    (None, 14, 14, 672)  0           ['normal_bn_1_17[0][0]']         
                                                                                                  
 separable_conv_1_normal_left1_  (None, 14, 14, 672)  468384     ['activation_237[0][0]']         
 17 (Separ

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x,
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [19]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=final_model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/NASNetLarge_YOLO', max_to_keep=3,
                                     checkpoint_name='NASNetLarge_YOLO')

In [20]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = final_model(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, final_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, final_model.trainable_weights))

In [21]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(tqdm(dataset)):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))

2023-11-21 20:01:57.775959, start training.


100%|██████████| 5482/5482 [10:42<00:00,  8.53it/s]


2023-11-21 20:12:40.428500, Epoch 1: loss 5.66
Saved checkpoint for epoch 1: ./ckpts/NASNetLarge_YOLO\NASNetLarge_YOLO-1


100%|██████████| 5482/5482 [10:35<00:00,  8.62it/s]


2023-11-21 20:23:19.004585, Epoch 2: loss 3.10
Saved checkpoint for epoch 2: ./ckpts/NASNetLarge_YOLO\NASNetLarge_YOLO-2


...

2023-11-22 03:08:07.572913, Epoch 38: loss 1.25
Saved checkpoint for epoch 38: ./ckpts/NASNetLarge_YOLO\NASNetLarge_YOLO-38


100%|██████████| 5482/5482 [10:35<00:00,  8.62it/s]


2023-11-22 03:18:45.959765, Epoch 39: loss 1.26
Saved checkpoint for epoch 39: ./ckpts/NASNetLarge_YOLO\NASNetLarge_YOLO-39


100%|██████████| 5482/5482 [10:35<00:00,  8.62it/s]


2023-11-22 03:29:24.658743, Epoch 40: loss 1.24
Saved checkpoint for epoch 40: ./ckpts/NASNetLarge_YOLO\NASNetLarge_YOLO-40


In [22]:
test_img_files = open('./pascal_voc_testing_data.txt')
test_img_dir = './VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []

for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])

test_dataset = tf.data.Dataset.from_tensor_slices(test_images)

def load_img_data(image_name):
    image_file = tf.io.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)

    h = tf.shape(image)[0]
    w = tf.shape(image)[1]

    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = (image/255) * 2 - 1

    return image_name, image, h, w

test_dataset = test_dataset.map(load_img_data, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [23]:
ckpt = tf.train.Checkpoint(net=final_model)
ckpt.restore('./ckpts/NASNetLarge_YOLO/NASNetLarge_YOLO-40')

In [24]:
output_file = open('./test_prediction.txt', 'w')

for img_name, test_img, img_h, img_w in test_dataset:
    batch_num = img_name.shape[0]
    for i in range(batch_num):
        xmin, ymin, xmax, ymax, class_num, conf = process_outputs(prediction_step(test_img[i:i+1]))
        xmin, ymin, xmax, ymax = xmin*(img_w[i:i+1]/IMAGE_SIZE), ymin*(img_h[i:i+1]/IMAGE_SIZE), xmax*(img_w[i:i+1]/IMAGE_SIZE), ymax*(img_h[i:i+1]/IMAGE_SIZE)

        #img filename, xmin, ymin, xmax, ymax, class, confidence
        output_file.write(img_name[i:i+1].numpy()[0].decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))

output_file.close()

In [26]:
import sys
sys.path.insert(0, './evaluate')

import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./test_prediction.txt', './output_file_NASNetLarge.csv')

import pandas as pd
cap = pd.read_csv('./output_file_NASNetLarge.csv')['packedCAP']
print('score: {:f}'.format(sum((1. - cap) ** 2) / len(cap)))

End Evalutation
score: 0.485389


## EfficientNetV2B3 + YOLO

In [ ]:
class DatasetGenerator:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """

    def __init__(self):
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(COM_DATA_PATH, 'r')

        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])

            self.record_list.append([float(num) for num in ss[1:]])

            self.object_num_list.append(
                min(len(self.record_list[-1]) // 5, MAX_OBJECTS_PER_IMAGE))
            if len(self.record_list[-1]) < MAX_OBJECTS_PER_IMAGE * 5:
                # if there are objects less than MAX_OBJECTS_PER_IMAGE, pad the list
                self.record_list[-1] = self.record_list[-1] + \
                    [0., 0., 0., 0., 0.] * \
                    (MAX_OBJECTS_PER_IMAGE - len(self.record_list[-1]) // 5)

            elif len(self.record_list[-1]) > MAX_OBJECTS_PER_IMAGE * 5:
               # if there are objects more than MAX_OBJECTS_PER_IMAGE, crop the list
                self.record_list[-1] = self.record_list[-1][:MAX_OBJECTS_PER_IMAGE * 5]

    def _data_preprocess(self, image_name, raw_labels, object_num):
        image_file = tf.io.read_file(COM_IMAGE_DIR + image_name)
        image = tf.io.decode_jpeg(image_file, channels=3)

        h = tf.shape(image)[0]
        w = tf.shape(image)[1]

        width_ratio = IMAGE_SIZE * 1.0 / tf.cast(w, tf.float32)
        height_ratio = IMAGE_SIZE * 1.0 / tf.cast(h, tf.float32)

        image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = (image/255) * 2 - 1
        image =  tf.keras.applications.efficientnet_v2.preprocess_input(image)

        raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)

        xmin = raw_labels[:, 0]
        ymin = raw_labels[:, 1]
        xmax = raw_labels[:, 2]
        ymax = raw_labels[:, 3]
        class_num = raw_labels[:, 4]

        xcenter = (xmin + xmax) * 1.0 / 2.0 * width_ratio
        ycenter = (ymin + ymax) * 1.0 / 2.0 * height_ratio

        box_w = (xmax - xmin) * width_ratio
        box_h = (ymax - ymin) * height_ratio

        labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis=1)

        return image, labels, tf.cast(object_num, tf.int32)

    def generate(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.image_names,
                                                      np.array(self.record_list),
                                                      np.array(self.object_num_list)))
        dataset = dataset.shuffle(100000)\
                         .map(self._data_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                         .batch(BATCH_SIZE, drop_remainder=True)\
                         .prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

In [ ]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
basemodel = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False, 
                                                                   weights='imagenet', 
                                                                   input_tensor=img_inputs,
                                                                   include_preprocessing=False)
basemodel.summary()

In [ ]:
basemodel.trainable = False
x = conv_leaky_relu(basemodel.output, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)

x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

In [ ]:
final_model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='EfficientNetV2B3_YOLO')
final_model.summary()

In [ ]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x,
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [ ]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=final_model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/EfficientNetV2B3_YOLO', max_to_keep=3,
                                     checkpoint_name='EfficientNetV2B3_YOLO')

In [ ]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = final_model(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, final_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, final_model.trainable_weights))

In [ ]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(tqdm(dataset)):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))

In [ ]:
test_img_files = open('./pascal_voc_testing_data.txt')
test_img_dir = './VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []

for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])

test_dataset = tf.data.Dataset.from_tensor_slices(test_images)

def load_img_data(image_name):
    image_file = tf.io.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)

    h = tf.shape(image)[0]
    w = tf.shape(image)[1]

    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = (image/255) * 2 - 1

    return image_name, image, h, w

test_dataset = test_dataset.map(load_img_data, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32)

In [ ]:
ckpt = tf.train.Checkpoint(net=final_model)
ckpt.restore('./ckpts/EfficientNetV2B3_YOLO/EfficientNetV2B3_YOLO-40')

In [ ]:
output_file = open('./test_prediction.txt', 'w')

for img_name, test_img, img_h, img_w in test_dataset:
    batch_num = img_name.shape[0]
    for i in range(batch_num):
        xmin, ymin, xmax, ymax, class_num, conf = process_outputs(prediction_step(test_img[i:i+1]))
        xmin, ymin, xmax, ymax = xmin*(img_w[i:i+1]/IMAGE_SIZE), ymin*(img_h[i:i+1]/IMAGE_SIZE), xmax*(img_w[i:i+1]/IMAGE_SIZE), ymax*(img_h[i:i+1]/IMAGE_SIZE)

        #img filename, xmin, ymin, xmax, ymax, class, confidence
        output_file.write(img_name[i:i+1].numpy()[0].decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))

output_file.close()

In [ ]:
import sys
sys.path.insert(0, './evaluate')

import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./test_prediction.txt', './output_file_EfficientNetV2B3.csv')

cap = pd.read_csv('./output_file_output_file_EfficientNetV2B3.csv')['packedCAP']
print('score: {:f}'.format(sum((1. - cap) ** 2) / len(cap)))